### A* em python

In [1]:
!pip install pygame
!python -m pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pygame
import math
from queue import PriorityQueue

In [4]:
import os 
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [5]:
WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption('A* IC TADS UFRN')

In [6]:
#cores
RED = (255,0,0)
GREEN = (0,255,0) #se está no conjunto aberto (open set)
BLUE = (0,0,255)
YELLOW = (255,255,0)
WHITE = (255,255,255)
BLACK = (0,0,0) #obstáculo
PURPLE = (128,0,128)
ORANGE = (255,165,0) #nó inicial
GRAY = (128,128,128)
TURQUOISE = (64,224,208)


In [7]:
# estado do nó
class Node:
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = WHITE #ainda não foi visitado
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows
    
    def get_pos(self):
        return self.row, self.col
    
    def is_closed(self):
        return self.color == RED #se já foi visitado
    
    def is_open(self):
        return self.color == GREEN #se está no open set
    
    def is_obstacle(self):
        return self.color == BLACK
    
    def is_start(self):
        return self.color == ORANGE
    
    def is_end(self):
        return self.color == PURPLE
    
    def reset(self):
        self.color = WHITE
    
    def make_start(self):
        self.color = ORANGE
    
    def make_closed(self):
        self.color = RED
        
    def make_open(self):
        self.color = GREEN
        
    def make_obstacle(self):
        self.color = BLACK
        
    def make_end(self):
        self.color = TURQUOISE
        
    def make_path(self):
        self.color = PURPLE
        
    def draw(self, win):
      
        pygame.draw.rect(win, self.color, pygame.Rect(self.x, self.y, self.width, self.width))
        
    def update_neighbors(self, grid):
        self.neighbors = []*self.col #apenas ao redor, sem diagonais
        #abaixo
        if self.row < self.total_rows-1 and not grid[self.row+1][self.col].is_obstacle():
            self.neighbors.append(grid[self.row+1][self.col])
        #acima
        if self.row > 0 and not grid[self.row-1][self.col].is_obstacle():
            self.neighbors.append(grid[self.row-1][self.col])
        #esquerda
        if self.col < self.total_rows-1 and not grid[self.row][self.col+1].is_obstacle():
            self.neighbors.append(grid[self.row][self.col+1])
        #direita
        if self.col > 0 and not grid[self.row][self.col-1].is_obstacle():
            self.neighbors.append(grid[self.row][self.col-1])
            
        #diagonal abaixo direita    
        if self.row < self.total_rows-1 and self.col > 0 and not grid[self.row+1][self.col-1].is_obstacle():
            self.neighbors.append(grid[self.row+1][self.col-1])
        #diagonal abaixo esquerda   
        if self.row < self.total_rows-1 and self.col < self.total_rows-1 and not grid[self.row+1][self.col+1].is_obstacle():
            self.neighbors.append(grid[self.row+1][self.col+1]) 
            
        #diagonal acima direita    
        if self.row > 0 and self.col > 0 and not grid[self.row-1][self.col-1].is_obstacle():
            self.neighbors.append(grid[self.row-1][self.col-1])
        #diagonal acima esquerda   
        if self.row > 0 and self.col < self.total_rows-1 and not grid[self.row-1][self.col+1].is_obstacle():
            self.neighbors.append(grid[self.row-1][self.col+1])   
    
    
    def __lt__(self, other):
        return False

In [8]:
#heuristica componente h - distância aproximada de um ponto a outro
import numpy as np
def h(p1, p2):
    #distancia manhatan (taxi - distancia L) (poderia ser euclidiana, etc.)
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt(pow(x1 - x2,2) + pow(y1 - y2,2))
    #return abs(x1 - x2) + abs(y1 - y2)

In [9]:
def reconstruct_path(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()
    

In [10]:
def a_star(draw, grid, start, end):
    count = 0 #tie brake
    open_set = PriorityQueue() #fila de prioridade
    #insere nó inicial com F = 0 (lembrar que f = g + h)
    open_set.put((0, count, start))
    came_from = {} #caminho inverso de onde veio para chegar ao nó atual
    g_score = {node: float("inf") for row in grid for node in row}
    g_score[start] = 0
    f_score = {node: float("inf") for row in grid for node in row}
    f_score[start] = h(start.get_pos(), end.get_pos())
    
    open_set_hash = {start} #para checar se elemento está na fila de prioridade
    
    while not open_set.empty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                
        current = open_set.get()[2]
        open_set_hash.remove(current)
        
        if current == end:
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True #construir caminho - chegou ao fim
        
        for neighbor in current.neighbors:
            temp_g_score = g_score[current] + 1 #assume arestas com 1
            if temp_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = temp_g_score
                f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open() #green
                    
        draw()
        
        if current != start:
            current.make_closed()

In [11]:
#construção da grade com certo número de linhas e certa largura
def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            node = Node(i, j, gap, rows)
            grid[i].append(node)
    return grid
    

In [12]:
#desenha as linhas na grade
def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GRAY, (0, i * gap), (width, i * gap))
        for j in range(rows):
            pygame.draw.line(win, GRAY, (j * gap, 0), (j * gap, width))

In [13]:
#deseja de fato a grade
def draw(win, grid, rows, width):
    win.fill(WHITE)
    for row in grid:
        for node in row:
            node.draw(win)
    draw_grid(win, rows, width)
    pygame.display.update()

In [14]:
#obtém posição do click de cada nó
def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y, x = pos
    row = y // gap
    col = x // gap
    return row, col

In [15]:
#main loop
def main(win, width):
    ROWS = 50
    grid = make_grid(ROWS, width)
    
    start = None
    end = None
    
    run = True
    started = False
    
    while run:
        draw(win, grid, ROWS, width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        
            if started:
                continue
            
            if pygame.mouse.get_pressed()[0]: #0 botão esquerdo do mouse
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                node = grid[row][col]
                if not start and node != end:
                    start = node
                    start.make_start()
                elif not end and node != start:
                    end = node
                    end.make_end()
                elif node != end and node != start:
                    node.make_obstacle()
            elif pygame.mouse.get_pressed()[2]: #0 botão direito do mouse
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                node = grid[row][col]
                node.reset()
                if node == start:
                    start = None
                elif node == end:
                    end = None
                    
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and not started:
                    for row in grid:
                        for node in row:
                            node.update_neighbors(grid)
                            
                a_star(lambda: draw(win, grid, ROWS, width), grid, start, end)
                
    pygame.quit()

In [ ]:
main(WIN, WIDTH)